# LangChain 모듈 - Model I/O
Language Model 애플리케이션의 핵심 요소는 Model입니다.   
LangChain은 언어모델(Language Model)과 인터페이스할 수 있는 빌딩블록을 제공합니다.
    
- Code 출처 : https://python.langchain.com/docs/modules/model_io/
- 수정사항 : 설명과 프롬프트 내용을 한글로 변경 

In [1]:
%pip install langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"

### Models
- LangChain이 통합하는 모델 유형으로 LLM과 ChatModel이 있습니다.
- LLM 객체는 문자열을 입력으로 받고 문자열을 반환합니다.
- ChatModel 객체는 메시지 리스트를 입력으로 받고 메시지 리스트를 반환합니다.

In [3]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

In [4]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

In [5]:
llm.invoke(text)

'\n\n"Rainbow Toes"'

In [6]:
chat_model.invoke(messages)

AIMessage(content='RainbowSock Co.')

### Prompt Templates
- 대부분의 LLM 애플리케이션은 사용자 입력을 LLM에 직접 전달하지 않습니다.
- 일반적으로 사용자 입력을 Prompt Template이라고 하는 더 큰 텍스트에 추가하여, 특정 작업에 대한 추가 컨텍스트를 제공합니다.
  
- PromptTemplate을 사용하여 문자열 프롬프트에 대한 템플릿을 생성합니다.
- 기본적으로 PromptTemplate은 템플릿 작성에 Python의 str.format 구문을 사용합니다.

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

템플릿은 변수가 없는 경우를 포함하여 원하는 갯수의 변수를 지원합니다:

In [9]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

- Prompt Template은 메시지 리스트를 만드는 데에도 사용할 수 있습니다.

### ChatPromptTemplate
- Chat Model 입력은 채팅 메시지 리스트입니다.
- 각 채팅 메시지는 콘텐츠 및 역할이라는 추가 매개변수와 연결됩니다.
- 예를 들어 OpenAI Chat Completions API에서 채팅 메시지는 AI assistant, human, system role에 연결될 수 있습니다.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="Bob", user_input="What is your name?")

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

- ChatPromptTemplate.from_messages는 다양한 메시지 표현을 허용합니다.
- 예를 들어 위에서 사용한 (type, content)의 2-tuple 표현을 사용하는 것 외에도 MessagePromptTemplate 또는 BaseMessage의 인스턴스를 전달할 수 있습니다.

In [36]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI


chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 사용자의 입력 문장을 아주 멋있게 긴 장문으로 새로 작성하는 소설 작가이다."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(
    text="하늘은 파랗게 구름은 하얗게 실바람도 불어와 부풀은 내 마음"
)
print(messages)

[SystemMessage(content='너는 사용자의 입력 문장을 아주 멋있게 긴 장문으로 새로 작성하는 소설 작가이다.'), HumanMessage(content='하늘은 파랗게 구름은 하얗게 실바람도 불어와 부풀은 내 마음')]


In [38]:
# Get a chat completion from the formatted messages
model = ChatOpenAI()
result = model.invoke(messages)
print(result)

content='그 순간, 나는 너와 함께 있을 때의 행복한 기억들이 내 마음을 가득 채우는 것을 느꼈다. 파란 하늘과 하얀 구름이 우리의 사랑스러운 이야기를 담아 흐르고, 부는 바람은 그 마음을 부풀리며 더 깊이 간직하고 싶다는 강한 욕망을 안겨주었다. 내 안의 감정은 무수히 많은 단어들로 표현하기 어려운 크고 깊은 바다처럼 넓고 깊어져가고 있었다. 머릿속에는 단어보다 더 아름다운 이미지가 펼쳐져 있었고, 눈앞에는 우리가 함께한 행복한 순간들이 자연스레 떠올랐다.\n\n그 순간, 우리의 사랑은 하늘과 구름과 바람과 함께 우리를 감싸 안았다. 파란 하늘은 우리의 미래를 밝고 푸르게 비추어 주었고, 하얀 구름은 우리의 사랑을 순수하고 아름답게 담아 주었다. 실바람은 우리의 마음을 더욱 깊이 뒤덮으며, 뜨거운 열정과 사랑이 불어와 우리를 더욱 가깝게 이어 주었다. 내 마음은 너와 함께 있을 때 더 빛을 발하고, 더욱 풍부해지며, 더욱 소중하게 느껴졌다.\n\n그 순간, 우리의 사랑은 시간과 공간을 초월하여 영원히 이어질 것이라는 확신이 내 마음을 충만하게 했다. 파란 하늘과 하얀 구름, 부는 실바람은 우리의 사랑을 지켜주고, 우리의 마음을 한결 깊게 이어주었다. 이 순간이 영원히 계속되길 바라며, 나는 너와 함께라면 어떤 어려움도 극복할 수 있다는 강한 확신으로 마음을 다잡았다. 이 순간이 끝나지 않기를, 우리의 사랑이 영원토록 이어지기를 진심으로 바라며, 내 마음을 파란 하늘과 하얀 구름, 부는 실바람과 함께 네게 닿도록 풀어 나가고 있었다.'


### Chat prompt composition
- LangChain은 프롬프트의 여러 부분을 함께 구성할 수 있는 사용자 친화적인 인터페이스를 제공합니다.
- 문자열 프롬프트 또는 채팅 프롬프트를 사용하여 이 작업을 수행할 수 있습니다.
- 이러한 방식으로 프롬프트를 구성하면 구성 요소를 쉽게 재사용할 수 있습니다.  

#### String prompt composition
- 문자열 프롬프트로 작업할 때 각 템플릿이 함께 결합됩니다.  
- 프롬프트를 직접 사용하거나 문자열을 사용하여 작업할 수 있습니다(리스트의 첫 번째 요소는 프롬프트여야 함).

In [71]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

In [72]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=prompt)
chain.run(topic="sports", language="korean")

'Why did the soccer player bring string to the game?\n\nBecause he heard they were playing "tug of war"! \n\nKorean: 축구 선수가 왜 줄을 가져갔을까요? "줄다리기"를 하는줄 알았기 때문에!'

#### Chat prompt composition
- Chat prompt 메시지 리스트로 구성됩니다
- 개발자 경험을 위해 이러한 프롬프트를 생성하는 편리한 방법을 추가 되었습니다.
- 이 파이프라인에서 각각의 새 요소는 최종 프롬프트의 새 메시지입니다.

In [29]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

LangChain에는 역할에 따라 다른 Message 클래스가 있습니다.
- HumanMessage : 사용자가 보낸 메시지
- AIMessage : 모델의 메시지
- SystemMessage : 모델에 동작 방법을 알려줍니다. 
- FunctionMessage : 함수 호출의 결과를 나타냅니다
- ToolMessage : 도구 호출의 결과를 나타냅니다.  
 
먼저 SystemMessage로 기본 ChatPromptTemplate을 초기화해 보겠습니다.  
SystemMessage로 시작할 필요는 없지만 다음과 같이 시작하는 것이 좋습니다.

In [31]:
prompt = SystemMessage(content="You are a nice pirate")

In [32]:
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

내부적으로는 ChatPromptTemplate 클래스의 인스턴스가 생성되므로 이전과 마찬가지로 사용할 수 있습니다.

In [33]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

 결과
이전과 마찬가지로 LLMChain에서도 사용할 수 있습니다.

In [35]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=new_prompt)
chain.run("나는 안녕이라고 말했다")

'안녕하세요! 어떻게 도와드릴까요?'

### ChatModel
- ChatModel은 LangChain의 핵심 구성 요소입니다.  
- LangChain은 OpenAI, Cohere, Hugging Face 등에서 제공하는 많은 Model과 상호 작용할 수 있는 표준 인터페이스를 제공합니다.
- LangChain을 사용하면 동기, 비동기, 배치(batch) 및 스트리밍 모드에서 모델을 사용할 수 있으며 캐싱 둥 기타 기능을 제공합니다.

In [43]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

In [44]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="모델 정규화의 목적은 무엇인가요?"),
]

In [45]:
chat.invoke(messages)

AIMessage(content='모델 정규화의 주요 목적은 모델의 성능을 개선하고 오버피팅을 방지하는 것입니다. 모델이 훈련 데이터에 너무 맞춰져서 새로운 데이터에 대해 일반화되지 못하는 오버피팅 문제를 해결하기 위해 사용됩니다. 모델 정규화는 가중치의 크기를 제한하거나 페널티를 부여하여 모델의 복잡도를 제어함으로써 오버피팅을 방지합니다. 이를 통해 모델이 새로운 데이터에 대해 더 일반화된 예측을 수행할 수 있도록 도와줍니다.')

In [22]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

모델 정규화의 주요 목적은 모델의 성능을 향상시키고, 과적합을 방지하여 모델의 일반화 성능을 개선하는 것입니다. 모델 정규화는 모델의 복잡도를 감소시키고, 불필요한 변수들의 영향을 줄여서 모델이 더욱 일반화된 결과를 얻을 수 있도록 도와줍니다. 이를 통해 모델이 새로운 데이터에 대해 더 정확하게 예측하고, 안정적인 성능을 유지할 수 있게 됩니다.

In [23]:
chat.batch([messages])

[AIMessage(content='모델 정규화의 주요 목적은 과적합(overfitting)을 방지하고 모델의 일반화 성능을 향상시키는 것입니다. 과적합은 모델이 훈련 데이터에 너무 맞춰져 새로운 데이터에 대해 일반화하지 못하는 현상을 말합니다. 모델 정규화는 이를 방지하기 위해 모델의 복잡도를 줄이거나 가중치를 제한하는 등의 방법을 사용하여 모델을 일반화할 수 있도록 도와줍니다. 이를 통해 모델이 새로운 데이터에 대해 더 정확하게 예측할 수 있도록 도와줍니다.')]

In [24]:
await chat.ainvoke(messages)

AIMessage(content='모델 정규화의 목적은 모델의 성능을 향상시키고 과적합을 줄이는 것입니다. 모델이 훈련 데이터에 너무 적합해지면 테스트 데이터나 실제 데이터에 대한 일반화 능력이 떨어질 수 있기 때문에, 정규화는 모델의 복잡성을 줄여 일반화 성능을 향상시키는 방법 중 하나입니다. 이를 통해 모델이 더 일반적인 패턴을 학습하고 새로운 데이터에 대해 더 잘 일반화할 수 있게 됩니다.')

In [25]:
async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)

모델 정규화의 주요 목적은 과적합(overfitting)을 방지하고 모델의 일반화 성능을 향상시키는 것입니다. 모델이 훈련 데이터에 너무 맞춰져서 새로운 데이터에 대해 일반화되지 못하는 과적합 문제를 해결하기 위해 모델의 복잡도를 줄이는 것이 중요합니다. 모델 정규화는 이를 달성하기 위해 모델의 파라미터를 제한하거나 조절하는 방법을 말합니다. 주요한 모델 정규화 기법으로는 L1, L2 규제, 드롭아웃, 배치 정규화 등이 있습니다.

In [26]:
async for chunk in chat.astream_log(messages):
    print(chunk)

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'c3c7c7d2-5992-406e-b205-7233c8b9d249',
            'logs': {},
            'name': 'ChatOpenAI',
            'streamed_output': [],
            'type': 'llm'}})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='모')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='모')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content='델')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='모델')})
RunLogPatch({'op': 'add',
  'path': '/streamed_output/-',
  'value': AIMessageChunk(content=' 정')},
 {'op': 'replace',
  'path': '/final_output',
  'value': AIMessageChunk(content='모델 정

#### Caching 
LangChain은 Chat Model을 위한 캐싱 레이어를 옵션으로 제공합니다. 
- 동일한 completion을 여러 번 요청하는 경우 LLM 제공업체에 대한 API 호출 횟수를 줄여 비용을 절감할 수 있습니다.
- LLM 제공업체에 대한 API 호출 횟수를 줄임으로써 애플리케이션 속도를 높일 수 있습니다.

In [46]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [47]:
%%time
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

D:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: total: 46.9 ms
Wall time: 915 ms


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [48]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: total: 0 ns
Wall time: 1 ms


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

#### SQLite Cache

In [49]:
# !rm .langchain.db
!del .langchain.db

In [50]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [51]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

CPU times: total: 31.2 ms
Wall time: 1.04 s


'Why did the scarecrow win an award? Because he was outstanding in his field!'

In [52]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: total: 0 ns
Wall time: 3 ms


'Why did the scarecrow win an award? Because he was outstanding in his field!'

#### Function calling 
- OpenAI와 같은 특정 Chat Model에는 function-calling API가 있으며, Model이 호출할 함수와 해당 함수에 대한 입력이 포함된 JSON 객체를 반환하도록 할 수 있습니다.
- Function-calling은 도구를 사용하여 Chain 및 Agent를 구성하고, 보다 일반적으로 Model에서 구조화된 출력을 얻는 데 매우 유용합니다.
- LangChain은 Function-calling을 쉽게 해주는 다양한 유틸리티를 제공합니다.
  1. 함수를 모델에 바인딩하기 위한 간단한 구문
  2. 다양한 유형의 객체를 예상되는 함수 스키마로 포맷하기 위한 변환기
  3. API 응답에서 함수 호출을 추출하기 위한 출력 파서

##### Python function

In [53]:
import json

from langchain_core.utils.function_calling import convert_to_openai_tool


def multiply(a: int, b: int) -> int:
    """Multiply two integers together.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b


print(json.dumps(convert_to_openai_tool(multiply), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "type": "integer",
          "description": "First integer"
        },
        "b": {
          "type": "integer",
          "description": "Second integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


##### Pydantic class

In [54]:
from langchain_core.pydantic_v1 import BaseModel, Field


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


print(json.dumps(convert_to_openai_tool(multiply), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "description": "First integer",
          "type": "integer"
        },
        "b": {
          "description": "Second integer",
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


##### LangChain Tool

In [55]:
from typing import Any, Type

from langchain_core.tools import BaseTool


class MultiplySchema(BaseModel):
    """Multiply tool schema."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseTool):
    args_schema: Type[BaseModel] = MultiplySchema
    name: str = "multiply"
    description: str = "Multiply two integers together."

    def _run(self, a: int, b: int, **kwargs: Any) -> Any:
        return a * b


# Note: we're passing in a Multiply object not the class itself.
print(json.dumps(convert_to_openai_tool(Multiply()), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "description": "First integer",
          "type": "integer"
        },
        "b": {
          "description": "Second integer",
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


##### Binding functions : 정의한 함수를 모델에 전달 합니다.

In [56]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm.invoke("what's 5 times three", tools=[convert_to_openai_tool(multiply)])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UR1Vdq9GRLaFFsEzTn97503z', 'function': {'arguments': '{"a":5,"b":3}', 'name': 'multiply'}, 'type': 'function'}]})

##### 도구를 호출할 때마다 이 함수가 전달되도록 하려면 이 함수를 도구에 바인딩하면 됩니다:

In [57]:
llm_with_tool = llm.bind(tools=[convert_to_openai_tool(multiply)])
llm_with_tool.invoke("what's 5 times three")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UR1Vdq9GRLaFFsEzTn97503z', 'function': {'arguments': '{"a":5,"b":3}', 'name': 'multiply'}, 'type': 'function'}]})

#### tool_choice 매개변수를 사용하여 도구가 호출되도록 강제할 수도 있습니다.

In [58]:
llm_with_tool = llm.bind(
    tools=[convert_to_openai_tool(multiply)],
    tool_choice={"type": "function", "function": {"name": "multiply"}},
)

llm_with_tool.invoke(
    "don't answer my question. no do answer my question. no don't. what's five times four"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JjJV6sdcF7dIZ1LmTLp6OGhx', 'function': {'arguments': '{"a":5,"b":4}', 'name': 'multiply'}, 'type': 'function'}]})

#### ChatOpenAI 클래스에는 함수형 개체를 OpenAI 형식으로 변환하고 바인딩하는 작업을 처리하는 bind_tools 헬퍼 함수도 포함되어 있습니다.

In [40]:
llm_with_tool = llm.bind_tools([multiply], tool_choice="multiply")
llm_with_tool.invoke("what's 5 times three")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dCE0TwvgEk1jpREC2kx54Zqm', 'function': {'arguments': '{"a":5,"b":3}', 'name': 'multiply'}, 'type': 'function'}]})

#### Tracking token usage

In [59]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a funny joke")
    print(cb)

Tokens Used: 28
	Prompt Tokens: 12
	Completion Tokens: 16
Successful Requests: 1
Total Cost (USD): $4.9999999999999996e-05


### LLM
- LLM은 LangChain의 핵심 구성 요소입니다.
- LangChain은 OpenAI, Cohere, Hugging Face 등에서 제공하는 다양한 LLM과 상호 작용하기 위한 표준 인터페이스를 제공합니다.
  
- LLM은 랭체인 표현식 언어(LCEL)의 기본 빌딩 블록인 Runnable  인터페이스를 구현합니다.
- 즉, invoke, ainvoke, stream, astream, batch, abatch, astream_log 호출을 지원합니다.
- LLM은 문자열을 입력으로 받거나 List[BaseMessage] 및 PromptValue와 같이 문자열 프롬프트로 강제 변환할 수 있는 객체를 입력으로 받습니다.

In [60]:
from langchain_openai import OpenAI

llm = OpenAI()

In [61]:
llm.invoke(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
)

"\n\n1. 필립스 곡선(Phillips Curve) 이론\n- 1958년 뉴질랜드의 경제학자 윌리엄 필립스가 발표한 이론으로, 실업률과 물가 상승률 간의 역방향 관계를 설명한다. 실업률이 낮을수록 임금 상승률이 높아지며, 이는 기업의 생산비용 증가로 이어져 물가 상승률이 높아지는 것이다.\n\n2. 루카스의 이론(Lucas's Theory)\n- 로버트 루카스가 제시한 이론으로, 인플레이션 기대가 실제 인플레이션에 영향을 미치는 것을 강조한다. 즉, 소비자나 기업이 미래의"

In [44]:
for chunk in llm.stream(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
):
    print(chunk, end="", flush=True)



1. 필립스 곡선 (Phillips Curve)
필립스 곡선은 실업률과 인플레이션의 상관관계를 설명하는 이론으로, 뉴질랜드의 경제학자 윌리엄 필립스가 1958년 제시한 것이다. 필립스는 영국의 실업률과 임금 상승률 간의 관계를 분석한 결과, 실업률이 낮을수록 임금 상승률이 높아지는 것을 발견하였다. 이를 바탕으로 필립스는 실업률과 인플레이션 사이에는 역의 관계가 있다고 주장하였다. 즉, 실업률이 낮을수록 인플레이션은 높아지고, 실업률

In [45]:
await llm.ainvoke(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
)

'\n\n1. 필립스 곡선 (Phillips Curve) 이론\n- 뉴질랜드의 경제학자 유진 필립스가 제안한 이론으로, 실업률과 인플레이션의 역관계를 설명한다. 이 이론에 따르면 실업률이 높을수록 임금 상승률이 낮아지고, 이는 고용량 근로자들의 소득이 감소하고 소비가 감소하게 되어 경제가 둔화하게 된다. 따라서 실업률이 낮아지면 임금 상승률이 높아지고 소비가 증가하여 경제가 확장하게 된다는 것이 이론의 핵심 내용이다.\n\n2. 자연율 이론 (Natural Rate Theory)\n- 미국의'

In [46]:
async for chunk in llm.astream(
    "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
):
    print(chunk, end="", flush=True)



1. 필립스 곡선 (Phillips Curve) 이론
필립스 곡선은 뉴질랜드의 경제학자 윌리엄 필립스에 의해 제시된 이론으로, 실업률과 인플레이션 사이의 상관관계를 설명하는 모형이다. 필립스는 1958년 영국에서 직업을 잃은 사람들의 평균 임금과 실업률 사이의 관계를 분석한 결과, 실업률이 낮을수록 임금 상승률이 높아지는 것을 발견했다. 이를 바탕으로 필립스는 실업률과 인플레이션은 역의 관계에 있으며, 이를 통해 정부가 실업률을 줄이기 위해 인

In [47]:
await llm.abatch(
    [
        "실업률과 인플레이션의 관계에 대한 이론에는 어떤 것이 있나요?"
    ]
)

['\n\n1. 필립스 곡선 (Phillips Curve) 이론\n- 뉴질랜드의 경제학자 유진 필립스가 제안한 이론으로, 실업률과 인플레이션의 역관계를 설명한다. 이 이론에 따르면 실업률이 높을수록 임금 상승률이 낮아지고, 이는 고용량 근로자들의 소득이 감소하고 소비가 감소하게 되어 경제가 둔화하게 된다. 따라서 실업률이 낮아지면 임금 상승률이 높아지고 소비가 증가하여 경제가 확장하게 된다는 것이 이론의 핵심 내용이다.\n\n2. 자연율 이론 (Natural Rate Theory)\n- 미국의']

### Output parsers
- Output parser(출력 구문 분석기)로 언어모델의 텍스트 출력을 구조화 합니다.
- Output parser에는 다음과 같은 몇 가지 주요 유형이 있습니다:
- LLM의 텍스트를 구조화된 정보(예: JSON)로 변환
- ChatMessage를 단순한 문자열로 변환
- 메시지 외에 호출에서 반환된 추가 정보(예: OpenAI 함수 호출)를 문자열로 변환

In [62]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

In [63]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1555-10-14T23:51:31.885834Z, 0782-01-21T22:36:57.215523Z, 0942-12-23T07:32:06.372704Z\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n{question}\n\n{format_instructions}')

In [64]:
chain = prompt | OpenAI() | output_parser
output = chain.invoke({"question": "when was bitcoin founded?"})

print(output)

2009-01-03 18:15:05


#### Pydantic parser
- 사용자가 임의의 Pydantic Model을 지정하고 해당 스키마를 준수하는 출력을 위해 LLM을 쿼리할 수 있다.
- Pydantic은 이터 유효성 검사와 구조화된 데이터를 파싱하기 위한 라이브러리입니다. 

In [65]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

##### LCEL
- Output parser는 LangChain 표현식 언어(LCEL)의 기본 구성 요소인 실행 가능한 인터페이스를 구현합니다.
- 즉, invoke, ainvoke, stream, astream, batch, abatch, astream_log 호출을 지원합니다.
- Output parser는 문자열 또는 BaseMessage를 입력으로 받아들이며 임의의 유형을 반환할 수 있습니다.

In [66]:
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

Output parser를 수동으로 호출하는 대신 실행 가능한 시퀀스에 추가할 수도 있습니다:

In [67]:
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

모든 parser가 스트리밍 인터페이스를 지원하지만, 특정 parser만 부분적으로 파싱된 객체를 통해 스트리밍할 수 있는데, 이는 출력 유형에 따라 크게 달라지기 때문입니다.  
부분 객체를 구성할 수 없는 parser가는 단순히 완전히 파싱된 출력을 생성합니다.   
예를 들어 SimpleJsonOutputParser는 부분 출력을 통해 스트리밍할 수 있습니다:

In [68]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

In [69]:
list(json_chain.stream({"question": "누가 현미경을 누가 발명했나요?"}))

[{},
 {'answer': ''},
 {'answer': '안'},
 {'answer': '안토'},
 {'answer': '안토니'},
 {'answer': '안토니 반'},
 {'answer': '안토니 반 레'},
 {'answer': '안토니 반 레웨'},
 {'answer': '안토니 반 레웨른'},
 {'answer': '안토니 반 레웨른후'},
 {'answer': '안토니 반 레웨른후크'},
 {'answer': '안토니 반 레웨른후크가'},
 {'answer': '안토니 반 레웨른후크가 현'},
 {'answer': '안토니 반 레웨른후크가 현미'},
 {'answer': '안토니 반 레웨른후크가 현미경'},
 {'answer': '안토니 반 레웨른후크가 현미경을'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발명'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발명했습니다'},
 {'answer': '안토니 반 레웨른후크가 현미경을 발명했습니다.'}]